In [9]:
import sys
sys.path.append("/home/daniel/software/MEEN537/")

import transforms as tr
import numpy as np
from visualization import VizScene 
np.set_printoptions(precision=3, suppress=True)


## Homogeneous matrix made of only rotation about z-axis by amount $\frac{\pi}{4}$

Rotation by amount theta around the z-axis should give the following:
$$
\left[\begin{matrix}0.707 & -0.707 & 0 & 0\\ 0.707 & 0.707 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [4]:
T = tr.SE3(tr.rotz(np.pi/4))
print(T)

Homogeneous Transform:
⎡0.9999   -0.01371   0    0 ⎤
⎢                           ⎥
⎢0.01371   0.9999    0    0 ⎥
⎢                           ⎥
⎢   0        0      1.0   0 ⎥
⎢                           ⎥
⎣   0        0       0   1.0⎦


## Translation only along the x-axis by an amount $0.5$ should give the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0.5 \\0 & 1 & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [5]:
T = tr.SE3(p=[0.5, 0, 0])
print(T)

Homogeneous Transform:
⎡1.0   0    0   0.5⎤
⎢                  ⎥
⎢ 0   1.0   0    0 ⎥
⎢                  ⎥
⎢ 0    0   1.0   0 ⎥
⎢                  ⎥
⎣ 0    0    0   1.0⎦


# Checking the "inv" function

If we calculate a homogenous transform with a rotation of 45 degrees about the x-axis and a general translation ($[0.5, 0.25, 0.3]^T$) we get the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0.5\\0 & 0.707106781186548 & -0.707106781186547 & 0.25\\0 & 0.707106781186547 & 0.707106781186548 & 0.3\\0 & 0 & 0 & 1\end{matrix}\right]
$$

Then, we can multiply the original transform by its inverse to check out inverse function, we should get the following:
$$
\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1.0 & 0 & 0\\0 & 0 & 1.0 & 0\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In [11]:
T = tr.SE3(tr.rotx(np.pi/4), [0.5, 0.25, 0.3])
# print(T)

# now we can check if we implemented "inv" correctly:
check = T.inv() @ T
print("\n\n inv(T) @ T should give identity matrix:")
print(check)




 inv(T) @ T should give identity matrix:
Homogeneous Transform:
⎡1.0   0    0    0 ⎤
⎢                  ⎥
⎢ 0   1.0   0    0 ⎥
⎢                  ⎥
⎢ 0    0   1.0   0 ⎥
⎢                  ⎥
⎣ 0    0    0   1.0⎦


# Transform a point in one frame to another frame


## DH parameters combined for a single joint:

Remember that if we combine a rotation in z, translation in z, then translation in x, and rotation in x, we should get the same result as the book for following the DH convention to move from one frame (or joint) to another as follows: 

$$
\left[\begin{matrix}\cos{\left(\theta \right)} & - \sin{\left(\theta \right)} \cos{\left(\alpha \right)} & \sin{\left(\alpha \right)} \sin{\left(\theta \right)} & a \cos{\left(\theta \right)}\\\sin{\left(\theta \right)} & \cos{\left(\alpha \right)} \cos{\left(\theta \right)} & - \sin{\left(\alpha \right)} \cos{\left(\theta \right)} & a \sin{\left(\theta \right)}\\0 & \sin{\left(\alpha \right)} & \cos{\left(\alpha \right)} & d\\0 & 0 & 0 & 1\end{matrix}\right]
$$

In future homework, we'll implement representations of robot arms that include this transformation. But for the test values of DH parameters below, we can perform this sequence of operations manually.  

Assuming the following DH parameters for two joints:

$\theta_1 = \frac{\pi}{8}$, $d_1 = 0$, $a_1 = 0.3$, $\alpha_1 = \frac{\pi}{2}$

$\theta_2 = \frac{\pi}{4}$, $d_2 = 0$, $a_2 = 0.3$, $\alpha_2 = 0$


The resulting homogeneous transform describing the tip (or frame 2) relate to frame 0 would be:

$$
\left[\begin{matrix}0.653281482438188 & -0.653281482438188 & 0.38268343236509 & 0.473148304484842\\0.270598050073099 & -0.270598050073099 & -0.923879532511287 & 0.195984444731456\\0.707106781186548 & 0.707106781186548 & 0 & 0.212132034355964\\0 & 0 & 0 & 1.0\end{matrix}\right]
$$

In [13]:
# start by substituting the actual values for R and p and making a new SE3 object
# that describes the transformation from frame 0 to frame 1

# find the transformation from frame 0 to 1
T1_in_0 = (tr.SE3(tr.rotz(np.pi/8), [0, 0, 0]) @ 
            tr.SE3(tr.rotx(np.pi/2), [0.3,0,0]))

# do the same thing for frame 1 to frame 2
T2_in_1 = (tr.SE3(tr.rotz(np.pi/4), [0,0,0]) @ 
            tr.SE3(tr.rotx(0), [0.3, 0, 0]))

# now we can combine the two to get a transformation that describes frame 2 
# relative to frame 0
T2_in_0 = T1_in_0 @ T2_in_1

# printing the result
print(T2_in_0)


Homogeneous Transform:
⎡ 0.9998    -0.02056  0.0001879   0.5999  ⎤
⎢                                         ⎥
⎢ 0.02056    0.9994   -0.02741   0.008223 ⎥
⎢                                         ⎥
⎢0.0003757  0.02741    0.9996    0.0001127⎥
⎢                                         ⎥
⎣    0         0          0         1.0   ⎦


In [14]:
################# Problem 2-37 ################################
T01 = tr.SE3(np.eye(3), [0, 1, 1]) # No rotation, 1m in y and z

T02 = tr.SE3(np.eye(3), [-0.5, 1.5, 1.1]) # No rotation

T03 = tr.SE3(tr.rotx(np.pi) @ tr.rotz(-np.pi/2), [-0.5, 1.5, 3])

T23 = tr.SE3(tr.rotx(np.pi) @ tr.rotz(-np.pi/2), [0, 0, 1.9])

print("T01:")
print(T01)
print("T02:")
print(T02)
print("T03:")
print(T03)
print("T23:")
print(T23)

T01:
Homogeneous Transform:
⎡1.0   0    0    0 ⎤
⎢                  ⎥
⎢ 0   1.0   0   1.0⎥
⎢                  ⎥
⎢ 0    0   1.0  1.0⎥
⎢                  ⎥
⎣ 0    0    0   1.0⎦
T02:
Homogeneous Transform:
⎡1.0   0    0   -0.5⎤
⎢                   ⎥
⎢ 0   1.0   0   1.5 ⎥
⎢                   ⎥
⎢ 0    0   1.0  1.1 ⎥
⎢                   ⎥
⎣ 0    0    0   1.0 ⎦
T03:
Homogeneous Transform:
⎡ 0.9996    0.02741     0     -0.5⎤
⎢                                 ⎥
⎢-0.02737   0.9981   -0.0548  1.5 ⎥
⎢                                 ⎥
⎢-0.001502  0.05478  0.9985   3.0 ⎥
⎢                                 ⎥
⎣    0         0        0     1.0 ⎦
T23:
Homogeneous Transform:
⎡ 0.9996    0.02741     0      0 ⎤
⎢                                ⎥
⎢-0.02737   0.9981   -0.0548   0 ⎥
⎢                                ⎥
⎢-0.001502  0.05478  0.9985   1.9⎥
⎢                                ⎥
⎣    0         0        0     1.0⎦
